In [1]:
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
from itertools import islice

# Load data

In [2]:
df = pd.read_csv("data/data-shogi-player.txt")
print(len(df))
df.head(5)

6231


,Loser,Winner
0,22,83
1,39,27
2,47,105
3,1,7
4,45,81


# Run Stan

In [3]:
model = CmdStanModel(stan_file="model/model10-4.stan")

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /workdir/chap10/model/model10-4


In [4]:
set_players = set().union(df["Winner"].values).union(df["Loser"].values)
len(set_players)

166

In [5]:
data = {
    "N": len(set_players),
    "G": len(df),
    "LW": df.values
}

In [6]:
fit = model.sample(data=data, show_progress=True)

Chain 1 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]

Chain 2 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]


Chain 4 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]

Chain 3 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]


Chain 1 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]

Chain 3 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]


Chain 1 - warmup:   5%|▌         | 100/2000 [01:12<23:03,  1.37it/s]

Chain 3 - warmup:   5%|▌         | 100/2000 [01:18<24:54,  1.27it/s]


Chain 2 - warmup:  10%|█         | 200/2000 [02:18<20:57,  1.43it/s]

Chain 1 - warmup:  10%|█         | 200/2000 [02:29<22:11,  1.35it/s]


Chain 2 - warmup:  15%|█▌        | 300/2000 [03:18<18:56,  1.50it/s]


Chain 1 - warmup:  15%|█▌        | 300/2000 [03:38<20:32,  1.38it/s]

Chain 2 - warmup:  20%|██        | 400/2000 [04:14<16:54,  1.58it/s]

Chain 1 - warmup:  20%|██        | 400/2000 [04:27<17:27,  1.53it/s]


Chain 2 - warmup:  25%|██▌       | 500/2000 [05:00<14:36,  1.71it/s]

Chain 1 - warmu

# Show result

In [7]:
players = dict()
with open("data/data-shogi-player-name.txt") as f:
    for line in islice(f, 1, None):
        tmp = line.strip().split("\t")
        players[int(tmp[2])] = tmp[1]

In [8]:
pf = np.median(fit.stan_variable("mu"), axis=0)
s_pf = np.median(fit.stan_variable("s_pf"), axis=0)

In [9]:
for i in islice(np.argsort(-pf), 0, 10):
    print(i+1, players[i+1], pf[i], s_pf[i])

47 羽生善治 1.862235 0.8624324999999999
105 渡辺明 1.61106 1.0237
134 豊島将之 1.3299400000000001 1.00109
78 行方尚史 1.2948499999999998 0.7822555
65 郷田真隆 1.2757399999999999 0.977347
130 糸谷哲郎 1.24915 1.202395
52 佐藤康光 1.24294 0.9606975
53 森内俊之 1.173005 1.23116
77 久保利明 1.10779 0.875632
59 屋敷伸之 1.092345 0.792592


In [10]:
for i in islice(np.argsort(-s_pf), 0, 10):
    print(i+1, players[i+1], s_pf[i])

155 藤森哲也 1.2828650000000001
53 森内俊之 1.23116
130 糸谷哲郎 1.202395
86 松本佳介 1.201015
99 山本真也 1.19232
51 神崎健二 1.18268
116 横山泰明 1.178245
111 宮田敦史 1.17424
71 深浦康市 1.1652
60 藤原直哉 1.1396549999999999
